In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cd "/content/gdrive/My Drive/"

In [13]:
!ls "/content/gdrive/My Drive/"

'41078792_2272142149731635_5511588839258324992_n(1).jpg'
'Album photo.gslides'
 cat
'Colab Notebooks'
'DeepLearning (1).ipynb'
 DeepLearning.ipynb
 Digits.ipynb
 dog
'Dog&Cat.ipynb'
 eniso-server-master.zip
'eniso tr'
'Formulaire sans titre.gform'
 nlp-getting-started
'Nouveau document RTF.rtf'
'PROJET SEMESTRIEL.rar'
 received_730462934089642.jpeg
 RecognitionProject.ipynb
 Retinopathie.ipynb
 sample_submission.csv
'sampleSubmission.csv (1)'
'smarteniso copie.eps'
 smarteniso.eps
'test1 (1)'
 test.csv
 train
 train.csv
'trainLabels.csv (1)'
 Untitled0.ipynb
 Untitled1.ipynb
 Untitled2.ipynb
 Untitled3.ipynb
 Untitled4.ipynb
 Untitled5.ipynb
 Untitled6.ipynb


In [0]:
import pandas as pd


In [15]:
train = pd.read_csv("/content/gdrive/My Drive/nlp-getting-started/train.csv")
test = pd.read_csv("/content/gdrive/My Drive/nlp-getting-started/test.csv")
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [0]:
import re
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\b(?:(?:https?|ftp)://)?\w[\w-]*(?:\.[\w-]+)+\S*', ' ', text)
    text = ' '.join(tokenizer.tokenize(text))
    return text

train_prep = train.text.apply(lambda x: clean_text(x))
test_prep = test.text.apply(lambda x: clean_text(x))
train['text_prep'] = train_prep
test['text_prep'] = test_prep

In [17]:
  import spacy.cli
  spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import spacy
nlp = spacy.load("en_core_web_lg")


In [19]:
import numpy as np

with nlp.disable_pipes():
    train_vectors = np.array([nlp(text).vector for text in train.text_prep])
    test_vectors = np.array([nlp(text).vector for text in test.text_prep])

print(train_vectors.shape, test_vectors.shape)

(7613, 300) (3263, 300)


In [0]:
from keras import initializers, regularizers, constraints
from sklearn.model_selection import KFold, cross_val_score, train_test_split
X_train = train_vectors
y_train = train.target.to_numpy()

train_x, test_x, train_y, test_y = train_test_split(X_train, y_train, test_size=0.2, random_state=13)

In [0]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svc = SVC(kernel='rbf', C=60, probability=True, random_state=13)
rfc = RandomForestClassifier(n_estimators=100, random_state=13)
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.2, random_state=13)


In [24]:
vcf = VotingClassifier(estimators=[('svc', svc), ('rfc', rfc), ('gbc', gbc)], voting='soft')
vcf.fit(X_train, y_train)

VotingClassifier(estimators=[('svc',
                              SVC(C=60, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                                  gamma='scale', kernel='rbf', max_iter=-1,
                                  probability=True, random_state=13,
                                  shrinking=True, tol=0.001, verbose=False)),
                             ('rfc',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini'...
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                              

In [26]:
from sklearn.metrics import accuracy_score
preds = vcf.predict(test_vectors)
print(accuracy_score((vcf.predict(test_x)), test_y))
print(len(test['id']), len(preds))

0.9763624425476034
3263 3263


In [0]:
submission = pd.DataFrame(columns=['id', 'target'])
submission['id'] = test['id']
submission['target'] = preds
submission.to_csv('submission.csv', index=False)